<a href="https://colab.research.google.com/github/Leo-2017/DL101/blob/main/Pattern_Player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pattern Player 🎹

## **1. Introduction**

## 1. What business problem are you solving?
Too much stress and bullshit people have to experience from work and lockdown situation. When they’re are home, they can us their own designed wallpaper in different rooms to reduce stress and make them feel calm when they get home from work; 
Yet hiring a designer is too expensive; 

A program for non-coders to get started with machine learning development. The course contains flexible and powerful ML pipelines for computer vision, tabular, recommender systems and NLP problems

## 2. How can AI/ML technology help to solve that?
A trained language model can 
text generator <br> 
Text Generation in Natural Language Processing
<br>
self-supervised learning
<br>
A language model can predict the probability of the next word in the sequence, based on the words already observed in the sequence.

## 3. What is your AI project's objective?


The objective A language model can predict the probability of the next word in the sequence, based on the words already observed in the sequence.

The objective is to build a language model as a pattern design generator that helps user to figure out what kind of wallpaper they would like to have.


Our Solution is a Language Model that uses the GPT-2 model (pretrained on a dataset of 8 million web pages) as a foundation, and our dataset above to fine-tune the model and make it more applicable for our use case.

## 4. Who are the users of your AI application and how will they interact with it? 

## **2. Benchmark Study**

Similar existing AI solutions

Use Case 1 <br>
**Delle-mini** <br> 
https://huggingface.co/spaces/dalle-mini/dalle-mini <br>
https://www.craiyon.com/ <br>



# **3. Data**

## 1. Libraries & Dependencies needed

In [ ]:
!pip install -Uqq unpackai
# !pip install -q git+https://github.com/unpackai/unpackai
!pip install -Uqq transformers==4.10.2
!pip install -q datasets transformers[sentencepiece]

     |████████████████████████████████| 73 kB 2.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 976 kB 58.7 MB/s 
     |████████████████████████████████| 2.8 MB 27.3 MB/s 
     |████████████████████████████████| 596 kB 56.8 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 880 kB 65.5 MB/s 
     |████████████████████████████████| 362 kB 33.2 MB/s 
     |████████████████████████████████| 140 kB 72.2 MB/s 
     |████████████████████████████████| 1.1 MB 60.9 MB/s 
     |████████████████████████████████| 212 kB 61.6 MB/s 
     |████████████████████████████████| 127 kB 60.4 MB/s 
     |████████████████████████████████| 144 kB 75.7 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 73.4 MB/s 
     |████████████████████████████████| 1.1 MB 41.3 MB/s 
ERROR: pip's depende

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModel,
    pipeline,
    set_seed,
    Trainer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    AutoModelForSequenceClassification,
)
from unpackai.nlp import Textual, InterpEmbeddingsTokenizer
from ipywidgets import interact
import logging
from fastai import *

## 2: Build an Language Model

### Step One - Define a ML problem and propose a solution.

#### 1. Define the objective
**Your objective:** Build a Language Model than can create the text of a fiction novel.
#### 2. Describe your dataset
**Your dataset:** The Return of Sherlock Holmes by Arthur Conan Doyle. 107,148 words. Gutenberg txt file.
#### 3. Describe your model
**Your model:** Transformer Model to generate text it was pretrained on, based on an initial set of words that I provide.

### Step Two - Collect and construct your dataset

In order to collect and design your own dataset we provide you with the `Textual` scraping tool below. For that you have two options!

**Option 1: Collect text from an URL.**

Provide an URL to the textual tool and all the text will be collected. We recommend [Gutenberg Project](https://gutenberg.org/) as a source which is a collection of over 60,000 free eBooks. Simply click on a book that you like and the corresponding txt file (see image below).

<img src="https://www.dropbox.com/s/p5tyklp58d3yosa/Screen%20Shot%202021-09-13%20at%2017.49.52.png?dl=1" alt="wordembeddings" width="500"/>

However, feel free to experiment with other datasets from other URL sources.


In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/36250/pg36250.txt")
textual

Text (393777 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/39749/pg39749.txt")
textual

Text (414489 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/17531/pg17531.txt")
textual

Text (80332 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/files/60034/60034-0.txt")
textual

Text (234063 chars), textual(),
    train_path, val_path = textual.create_train_val()

**Option 2: Collect text from path.**

In case you have a downloaded txt file, feel free to upload it to the Google Drive and enter the path in the bracket below. Make sure that the file format is txt (a type of file just like ppt or csv).

In [ ]:
textual = Textual.from_path("./the_txt_file_you_uploaded.txt")
textual

Traceback (most recent call last):
  File "<ipython-input-5-7af509265d33>", line 1, in <module>
    textual = Textual.from_path("./the_txt_file_you_uploaded.txt")
  File "/usr/local/lib/python3.7/dist-packages/unpackai/nlp/data.py", line 59, in from_path
    raise FileExistsError(f"Cannot find {path}")
FileExistsError: Cannot find the_txt_file_you_uploaded.txt



### Step Three - Model Training

We skip the Data Transformation as the Tokenization and Numericalisation happens during the Model Training.

In [ ]:
pretrained_model = pipeline("text-generation", model='gpt2')

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50

### Step Four - Interpret the model and generate text.

In [ ]:
arguments = TrainingArguments(
    output_dir="./write_style",
    overwrite_output_dir=True,  
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=64,
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = textual.get_hf_trainer(
    model=pretrained_model.model,
    tokenizer = pretrained_model.tokenizer,
    arguments = arguments
    )

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file ./cached_lm_GPT2TokenizerFast_128_train_text.txt [took 0.004 s]
Loading features from cached file ./cached_lm_GPT2TokenizerFast_128_val_text.txt [took 0.001 s]


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 573
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 72


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=72, training_loss=3.598634507921007, metrics={'train_runtime': 54.9901, 'train_samples_per_second': 31.26, 'train_steps_per_second': 1.309, 'total_flos': 112290250752000.0, 'train_loss': 3.598634507921007, 'epoch': 3.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to ./write_style
Configuration saved in ./write_style/config.json
Model weights saved in ./write_style/pytorch_model.bin


In [ ]:
style_writer = pipeline('text-generation',
                        model='./write_style',
                        tokenizer=pretrained_model.tokenizer
                        )

loading configuration file ./write_style/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 50,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.10.2",
  "use_cache": true,
  "vocab_size": 50257
}

loading configurati

In [ ]:
style_writer("How to decorate my wall with botanical ornaments?",
            max_length=200,
            num_return_sequences=1)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'How to decorate my wall with botanical ornaments? I take great pains to make it well-stocked with all kinds of furniture suitable for use. Many artists who have succeeded themselves in doing an artistic thing have brought home so many important objects that the beauty of the designs can hardly be denied--or can they be claimed to have done without any artistic intention at all? We can, of course, hardly understand the importance of these things if we are not interested in them. How much more important is they than that of furniture, with whom we must rely all our leisure and energy, when workmanship is as much necessary and indispensable to any artistic work as it was even sixteenth-century furniture!  Thus long as these people still do not learn enough to distinguish the art from the skill of carving, and to appreciate the artistic character of the parts they are attempting to imitate, they will soon find themselves unable to appreciate the qualities of the finish

In [ ]:
style_writer("How to decorate my bedroom wall with blossom pattern ornaments",
            max_length=200,
            num_return_sequences=3)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'How to decorate my bedroom wall with blossom pattern ornaments, and how will its ornament be affected by the style of the designer?     With the help of Dr. Kekri, I set out, on the 18th of May, from Strasbourg.      This period is especially important since the English Renaissance is the last in which the French and Italian churches are entirely devoted to the study of painting.   Our little studio in the neighbourhood of Strasbourg was the object of the study of two Italian artistes: Lorenzo Maggio, and Maria Corrado, and the works of her sister-in-law Francesco Aarion da Figaro-Coppini, both in fine details and in a rich, Gothic pattern, were also displayed, in the chapel of the Christian Church.     Now the Gothic work of Giuseppe Veneto has been adapted, and the Gothic art of Milan and Paris has been rendered'},
 {'generated_text': 'How to decorate my bedroom wall with blossom pattern ornaments, as my friend J. H. Johnson writes, is an essential craft for this

In [ ]:
style_writer("How to decorate my bedroom wall with cyclamen persicum pattern?",
            max_length=200,
            num_return_sequences=1)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'How to decorate my bedroom wall with cyclamen persicum pattern?   I will show you how to do it, and I\'ll give you a little explanation, for myself, as well as a picture which seems almost necessary to illustrate the task. I have three examples--one of these a small and another very large--and I am the first to see that it will be in style as well as useful, which the first of the three is quite clear in its simplicity. For the first-mentioned, as has been mentioned, it is a kind of lace, as all lace has its characteristic colours of pink, a crimsonish hue, or red, the colour of which is the first thing you see--this is the colour most suited to lace-work, of course, by itself; but then, of course, what to call it is an artificial or "strict" lace, being a material of very thin and unkempt threads, like that of some furniture.'}]